In [19]:
import os
os.chdir(r"c:\Users\rohan\Documents\Flutter Projects\saas-subscription-analytics")
os.getcwd()

'c:\\Users\\rohan\\Documents\\Flutter Projects\\saas-subscription-analytics'

In [20]:
import pandas as pd

accounts = pd.read_csv("data/processed/accounts_cleaned.csv")
subscriptions_cleaned = pd.read_csv("data/processed/subscriptions_cleaned.csv")
churn_events_cleaned = pd.read_csv("data/processed/churn_events_cleaned.csv")
feature_usage_cleaned = pd.read_csv("data/processed/feature_usage_cleaned.csv")
support_tickets_cleaned = pd.read_csv("data/processed/support_tickets_cleaned.csv")

In [21]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Convert date fields to datetime where needed
subscriptions_cleaned["start_date"] = pd.to_datetime(subscriptions_cleaned["start_date"])
subscriptions_cleaned["end_date"] = pd.to_datetime(subscriptions_cleaned["end_date"])
feature_usage_cleaned["usage_date"] = pd.to_datetime(feature_usage_cleaned["usage_date"])
support_tickets_cleaned["submitted_at"] = pd.to_datetime(support_tickets_cleaned["submitted_at"])
support_tickets_cleaned["closed_at"] = pd.to_datetime(support_tickets_cleaned["closed_at"])

# Prepare revenue time series
mrr_ts = (
    subscriptions_cleaned
    .groupby(subscriptions_cleaned["start_date"].dt.to_period("M"))["mrr_amount"]
    .sum()
    .to_timestamp()
)

arr_ts = mrr_ts * 12

#Properly Convert Churn Date to Datetime
churn_events_cleaned["churn_date"] = pd.to_datetime(
    churn_events_cleaned["churn_date"], errors="coerce"
)


# Prepare churn time series
churn_ts = (
    churn_events_cleaned
    .groupby(churn_events_cleaned["churn_date"].dt.to_period("M"))["account_id"]
    .count()
    .to_timestamp()
)

# Feature usage aggregation
feature_daily = (
    feature_usage_cleaned
    .groupby(feature_usage_cleaned["usage_date"].dt.date)["usage_count"]
    .sum()
)

feature_heatmap = (
    feature_usage_cleaned
    .groupby(["feature_name", feature_usage_cleaned["usage_date"].dt.to_period("M")])["usage_count"]
    .sum()
    .unstack(fill_value=0)
)

# Support tickets timeline
tickets_ts = (
    support_tickets_cleaned
    .groupby(support_tickets_cleaned["submitted_at"].dt.to_period("M"))["ticket_id"]
    .count()
    .to_timestamp()
)


In [22]:
fig = px.line(
    mrr_ts,
    title="MRR Trend Over Time",
    labels={"value": "MRR ($)", "index": "Month"},
)
fig.update_layout(yaxis_tickformat="$,.0f")
fig.show()


In [23]:
fig = px.line(
    arr_ts,
    title="ARR Trend Over Time",
    labels={"value": "ARR ($)", "index": "Month"},
)
fig.update_layout(yaxis_tickformat="$,.0f")
fig.show()


In [24]:
fig = px.bar(
    churn_ts,
    title="Monthly Churn Events",
    labels={"value": "Churn Count", "index": "Month"},
)
fig.show()


In [25]:
gross_revenue = mrr_ts.iloc[-1]
expansion = 1262997
contraction = 459366
churn_loss = gross_revenue * 0.10  # approx example if 10% churned revenue

net_revenue = gross_revenue + expansion - contraction - churn_loss

funnel = pd.DataFrame({
    "Stage": ["Gross Revenue", "Expansion", "Contraction", "Churn Loss", "Net Revenue"],
    "Value": [gross_revenue, expansion, -contraction, -churn_loss, net_revenue]
})

fig = px.funnel(
    funnel,
    x="Value",
    y="Stage",
    title="Revenue Retention Funnel (NRR Components)",
)
fig.update_layout(xaxis_tickformat="$,.0f")
fig.show()


In [26]:
feature_heatmap.index = feature_heatmap.index.astype(str)
feature_heatmap.columns = feature_heatmap.columns.astype(str)

fig = px.imshow(
    feature_heatmap,
    labels=dict(x="Month", y="Feature", color="Usage Count"),
    title="Feature Usage Heatmap",
    aspect="auto",
)
fig.show()


In [ ]:
fig = px.line(
    tickets_ts,
    title="Support Tickets Submitted Per Month",
    labels={"value": "Ticket Count", "index": "Month"},
)
fig.show()


: 